# Dendrite Phase Field Benchmark

FiPy implementation of 2D dendritic solidification benchmark.

https://pages.nist.gov/pfhub/benchmarks/benchmark3.ipynb/

## Import Python modules

In [ ]:
import json
import re
import sys
from scipy.interpolate import CubicSpline
from scipy.optimize import root_scalar

try:
    import pathlib
except ImportError:
    import pathlib2 as pathlib

import fipy as fp
from fipy.tools import numerix as nmx
from fipy.tools import parallelComm
from steppyngstounes import CheckpointStepper, FixedStepper

In [ ]:
isnotebook = False
try:
    from IPython import get_ipython
    isnotebook = (get_ipython().__class__.__name__ == "ZMQInteractiveShell")
except:
    pass

## Define problem

In [ ]:
param_file = pathlib.Path(sys.argv[1])
work_dir = param_file.parent

### Define simulation parameters

In [ ]:
if isnotebook:
    params = {
        "dx": 0.5,
        "Lx": 96.,
        "r0": 8.,
        "dt": 0.15,
        "t_max": 100,
        "checkpoints": nmx.linspace(0, 100, 11),
        "view": True,
        "solver": None,
        "preconditioner": None
    }
else:
    with open(param_file, "r") as f:
        params = json.load(f)

### Define domain

In [ ]:
mesh = fp.Grid2D(dx=params["dx"], Lx=params["Lx"],
                 dy=params["dx"], Ly=params["Lx"])

### Define solution variables

In [ ]:
u = fp.CellVariable(mesh=mesh, name="u", hasOld=True)

In [ ]:
phi = fp.CellVariable(mesh=mesh, name=r"$\varphi$", hasOld=True)

### Define equations

In [ ]:
D = 10.
tau0 = 1.
W0 = 1.
lamda = D * tau0 / (0.6267 * W0**2)
norm = phi.grad / (phi.grad.mag + (phi.grad.mag == 0.) * 1.)
norm_f = phi.faceGrad / (phi.faceGrad.mag + (phi.faceGrad.mag == 0.) * 1.)
theta = nmx.arctan2(norm[1], norm[0])
theta.name = r"$\theta$"
theta_f = nmx.arctan2(norm_f[1], norm_f[0])
theta_f.name = r"$\theta$"
m = fp.Variable(4)
epsilon = fp.Variable(0.05)
a = 1 + epsilon * nmx.cos(m * theta)
a_f = 1 + epsilon * nmx.cos(m * theta_f)
aPrime = -epsilon * nmx.sin(m * theta) * m
aPrime_f = -epsilon * nmx.sin(m * theta_f) * m
tau = tau0 * a**2
W = W0 * a
W_f = W0 * a_f
Wprime = W0 * aPrime
Wprime_f = W0 * aPrime_f
Dphi = W_f**2
Dphi = W_f * (W_f * [[1, 0],
                     [0, 1]] + Wprime_f * [[0, -1],
                                           [1, 0]])
Delta = fp.Variable(value=0.3, name=r"$\Delta$")

In [ ]:
ueq = (fp.TransientTerm(coeff=1., var=u) 
       == fp.DiffusionTerm(coeff=D, var=u)
       + fp.TransientTerm(coeff=0.5, var=phi))

In [ ]:
phieq = (fp.TransientTerm(coeff=tau, var=phi)
         == fp.DiffusionTerm(coeff=Dphi, var=phi)
         + fp.ImplicitSourceTerm(coeff=1-phi**2, var=phi)
         - fp.ImplicitSourceTerm(coeff=lamda * (1 - 2*phi**2 + phi**4), var=u))

In [ ]:
eq = ueq & phieq

### Define boundary conditions

In [ ]:
u.constrain(-Delta, where=mesh.facesRight | mesh.facesTop)

### Define metrics

In [ ]:
f_chem = (-(1/2) * phi**2 + (1/4) * phi**4
          + lamda * u * phi * (1 - (2/3) * phi**2 + (1/5) * phi**4))
free_energy = ((1/2) * W**2 * phi.grad.mag**2 + f_chem).cellVolumeAverage * nmx.sum(mesh.cellVolumes)
solid_fraction = ((phi + 1) / 2).cellVolumeAverage

In [ ]:
class TipVariable(fp.Variable):
    def __init__(self, var):
        fp.Variable.__init__(self)
        self.var = self._requires(var)
        mesh = var.mesh
        X = mesh.faceCenters[0, mesh.facesBottom]
        self.ix = nmx.argsort(X)
        self.X = X[self.ix]
        self.var_value = self.var.faceValue[mesh.facesBottom][self.ix]

    def _calcValue(self):
        spl = CubicSpline(self.X, self.var_value)
        sol = root_scalar(spl, bracket=(0, params["Lx"]))
        if sol.converged:
            return sol.root
        else:
            raise Exception

tip_position = TipVariable(phi)

In [ ]:
faceNorm = phi.faceGrad / (phi.faceGrad.mag + (phi.faceGrad.mag == 0.) * 1.)
faceNorm.name = r"$\hat{n}$"
curvature = faceNorm.divergence
curvature.name = r"$\kappa$"

In [ ]:
radius = 1. / curvature
radius.name = r"$\rho$"

### Initialize viewers

In [ ]:
if params["view"]:
    phiviewer = fp.Viewer(vars=phi, datamin=-1, datamax=1)

In [ ]:
# radviewer = fp.Viewer(vars=radius *(1 + phi) * (1-phi), datamin=-20, datamax=0)

In [ ]:
# radviewer.plot()

In [ ]:
# fp.tools.dump.write(mesh, "mesh.gz")

### Create solver

In [ ]:
precon = None

if params["preconditioner"] == "jacobi":
    precon = fp.JacobiPreconditioner()
elif params["preconditioner"] == "ilu":
    precon = fp.ILUPreconditioner()
elif params["preconditioner"] == "ssor":
    precon = fp.SSORPreconditioner()
elif params["preconditioner"] == "icc":
    precon = fp.ICPreconditioner()
elif params["preconditioner"] in [None, "none"]:
    precon = None
else:
    precon = fp.solvers.__dict__[params["preconditioner"]]()

if params["solver"] == "cgs":
    solver_class = fp.LinearCGSSolver
elif params["solver"] == "gmres":
    solver_class = fp.LinearGMRESSolver
elif params["solver"] == "lu":
    solver_class = fp.LinearLUSolver
elif params["solver"] == "pcg":
    solver_class = fp.LinearPCGSolver
elif params["solver"] in [None, "none"]:
    solver_class = None
else:
    solver_class = fp.solvers.__dict__[params["solver"]]

In [ ]:
if solver_class == fp.LinearLUSolver:
    precon = None # preconditioned lu doesn't make any sense

In [ ]:
if params["solver"] in [None, "none"]:
    solver = None
else:
    solver = solver_class(criterion="RHS", precon=precon)    

## Setup output

### Define output routines

In [ ]:
def checkpoint_data(elapsed):
    u_value = u.globalValue
    phi_value = phi.globalValue
    if parallelComm.procID == 0:
        fname = work_dir / "t={}.npz".format(elapsed)
        nmx.savez(fname,
                  phi=phi_value,
                  u=u_value)

### Figure out when to save

In [ ]:
checkpoints = nmx.asarray(params["checkpoints"])
checkpoints.sort()

## Initialize fields

In [ ]:
r = mesh.cellCenters.mag

In [ ]:
if params["restart"] is not None:
    data = nmx.load(params["restart"])
    u.value = data["u"]
    phi.value = data["phi"]

    # scanf("%g") simulator
    # https://docs.python.org/3/library/re.html#simulating-scanf
    scanf_g = r"[-+]?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?"
    pattern = r".*t=({g})\.npz".format(g=scanf_g)
    elapsed = float(re.match(pattern, params["restart"]).group(1))
else:
    elapsed = 0.
    phi.value = -1
    phi.setValue(+1, where=r <= params["r0"])
    u.value = -Delta

In [ ]:
if elapsed in checkpoints:
    checkpoint_data(elapsed=elapsed)

## Solve in time

In [ ]:
times = checkpoints
times = times[(times > elapsed)
              & (times <= params["t_max"])]

In [ ]:
print(f"t,free_energy,solid_fraction,tip_position")
print(f"{elapsed},{free_energy},{solid_fraction},{tip_position}")

for checkpoint in CheckpointStepper(start=elapsed,
                                    stops=times,
                                    stop=params["t_max"]):

    for step in FixedStepper(start=checkpoint.begin,
                             stop=checkpoint.end,
                             size=params["dt"]):

        phi.updateOld()
        u.updateOld()
        for sweep in range(3):
            res = eq.sweep(dt=step.size, solver=solver)

        if isnotebook and params["view"]:
            phiviewer.plot()
        print(f"{step.end},{free_energy},{solid_fraction},{tip_position}")

        _ = step.succeeded() # FixedStepper can't fail

    if checkpoint.end in checkpoints:
        checkpoint_data(elapsed=checkpoint.end)

    _ = checkpoint.succeeded() # CheckpointStepper can't fail                               